In [ ]:
import os

url_prefix = input('url_prefix:')
print(url_prefix)
print()

page_num = int(input('page_num:'))
page_num_len = len(str(page_num - 1))
print(page_num, page_num_len)
print()


book_name = input('book_name:')
print(book_name)
print()

dirname = os.path.join('Rakuten_download', book_name)
if not os.path.exists(dirname):
    os.makedirs(dirname)

pagenums = [str(x).zfill(page_num_len) for x in range(page_num)]
print(pagenums)

In [ ]:
def url_gen():
    for pagenum in pagenums:
        url = url_prefix + pagenum + '.pdf'
        yield url, pagenum + '.pdf'

In [ ]:
import threading
from urllib import request

urls = url_gen()
lock = threading.Lock()

def loop_download():
    while True:
        try:
            with lock:
                url, filename = next(urls)
        except:
            break
        try:
            print('Downloading ', url)
            with request.urlopen(url) as urlf:
                with open(os.path.join(dirname, filename), 'wb') as f:
                    f.write(urlf.read())
        except:
            print('Failed: ', url)
            exit()

In [ ]:
thread_num = 20
threads = []

for i in range(thread_num):
    t = threading.Thread(target=loop_download)
    threads.append(t)
    t.start()
    

for t in threads:
    t.join()

In [ ]:
import binascii

def get_jpg(filename_in, filename_out):
    infile = open(os.path.join(dirname, filename_in), 'rb')
    outfile = open(os.path.join(dirname, filename_out), 'wb')

    # 读取文件头部，并查找 JPEG 文件头
    hex_data = binascii.b2a_hex(infile.read(1500))
    str_sidx = bytes.decode(hex_data).find('ffd8ff')
    str_sidx = str_sidx // 2
    # 确定位置后，重置文件指针至文件头部
    infile.seek(0, 0)
    # 切割文件
    outfile.write(infile.read()[str_sidx:])
    infile.close()
    outfile.close() 

i = 0
while i < len(pagenums):
    if i == 0 or i == len(pagenums) - 1:
        pagenum = pagenums[i]
        filename_in = pagenum + '.pdf'
        filename_out = pagenum + '.jpg'
        get_jpg(filename_in, filename_out)
        i = i + 1
        continue
    else:
        j = i + 1
        pagenum = [pagenums[x] for x in [i, j]]
        filename_in = [x + '.pdf' for x in pagenum]
        filename_out = [x + '.jpg' for x in reversed(pagenum)]
        for x in range(2):
            get_jpg(filename_in[x], filename_out[x])
        i = i + 2

In [ ]:
import glob
import zipfile

for f in glob.glob(os.path.join(dirname, '*.pdf')):
     os.remove(f)
        
def make_zip(source_dir, output_filename):
    output_filename = output_filename + '.cbz'
    zipf = zipfile.ZipFile(output_filename, 'w')    
    pre_len = len(os.path.dirname(source_dir))
    for parent, dirnames, filenames in os.walk(source_dir):
        for filename in filenames:
            pathfile = os.path.join(parent, filename)
            arcname = pathfile[pre_len:].strip(os.path.sep)     #相对路径
            zipf.write(pathfile, arcname)
    zipf.close()
    print(output_filename)
    
make_zip(dirname, book_name)